In [2]:
#!/usr/bin/python
# coding=utf-8
# 采用TF-IDF方法提取文本关键词
# http://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting
import sys,codecs
import pandas as pd
import numpy as np
import jieba.posseg
import jieba.analyse
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
"""
       TF-IDF权重：
           1、CountVectorizer 构建词频矩阵
           2、TfidfTransformer 构建tfidf权值计算
           3、文本的关键字
           4、对应的tfidf矩阵
"""
# 数据预处理操作：分词，去停用词，词性筛选
def dataPrepos(text, stopkey):
    l = []
    pos = ['n', 'nz', 'v', 'vd', 'vn', 'l', 'a', 'd']  # 定义选取的词性
    seg = jieba.posseg.cut(text)  # 分词
    for i in seg:
        if i.word not in stopkey and i.flag in pos:  # 去停用词 + 词性筛选
            l.append(i.word)
    return l

# tf-idf获取文本top10关键词
def getKeywords_tfidf(data,stopkey,topK):
    idList, titleList, abstractList = data['id'], data['title'], data['abstract']
    corpus = [] # 将所有文档输出到一个list中，一行就是一个文档
    for index in range(len(idList)):
        text = '%s。%s' % (titleList[index], abstractList[index]) # 拼接标题和摘要
        text = dataPrepos(text,stopkey) # 文本预处理
        text = " ".join(text) # 连接成字符串，空格分隔
        corpus.append(text)

    # 1、构建词频矩阵，将文本中的词语转换成词频矩阵
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus) # 词频矩阵,a[i][j]:表示j词在第i个文本中的词频
    # 2、统计每个词的tf-idf权值
    transformer = TfidfTransformer()
    tfidf = transformer.fit_transform(X)
    # 3、获取词袋模型中的关键词
    word = vectorizer.get_feature_names()
    # 4、获取tf-idf矩阵，a[i][j]表示j词在i篇文本中的tf-idf权重
    weight = tfidf.toarray()
    # 5、打印词语权重
    ids, titles, keys = [], [], []
    for i in range(len(weight)):
        print (u"-------这里输出第", i+1 , u"篇文本的词语tf-idf------")
        ids.append(idList[i])
        titles.append(titleList[i])
        df_word,df_weight = [],[] # 当前文章的所有词汇列表、词汇对应权重列表
        for j in range(len(word)):
            print (word[j],weight[i][j])
            df_word.append(word[j])
            df_weight.append(weight[i][j])
        df_word = pd.DataFrame(df_word,columns=['word'])
        df_weight = pd.DataFrame(df_weight,columns=['weight'])
        word_weight = pd.concat([df_word, df_weight], axis=1) # 拼接词汇列表和权重列表
        word_weight = word_weight.sort_values(by="weight",ascending = False) # 按照权重值降序排列
        keyword = np.array(word_weight['word']) # 选择词汇列并转成数组格式
        word_split = [keyword[x] for x in range(0,topK)] # 抽取前topK个词汇作为关键词
        word_split = " ".join(word_split)
        keys.append(word_split)

    result = pd.DataFrame({"id": ids, "title": titles, "key": keys},columns=['id','title','key'])
    return result


def main():
    # 读取数据集
    dataFile = 'data/sample_data.csv'
    data = pd.read_csv(dataFile)
    # 停用词表
    stopkey = [w.strip() for w in codecs.open('data/stopWord.txt', 'r',encoding='utf-8').readlines()]
    # tf-idf关键词抽取
    result = getKeywords_tfidf(data,stopkey,10)
    result.to_csv("result/keys_TFIDF.csv",index=False)

if __name__ == '__main__':
    main()


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.575 seconds.
Prefix dict has been built succesfully.


-------这里输出第 1 篇文本的词语tf-idf------
一定 0.05493304800244877
一段时间 0.0
主体 0.0
乘客 0.0
乘车 0.0
事件 0.0
二者 0.0
交叉 0.0
产品 0.0
产生 0.0933962189683336
介绍 0.0
仪表板 0.0
传感器 0.05493304800244877
估计 0.0
位置 0.040855327621860726
使用 0.0
使能 0.05493304800244877
使该 0.0
信号 0.0933962189683336
倾斜 0.0
偏压 0.0
停转 0.0
储能 0.0
元件 0.0
关闭 0.0
具有 0.0
内燃机 0.0
内管 0.0
凹陷 0.0
分析 0.0
分部 0.0
分隔 0.0
切换 0.05493304800244877
利用 0.0
制动 0.10986609600489754
制造 0.0
刹车 0.05493304800244877
力矩 0.1647991440073463
功能 0.05493304800244877
包围 0.0
单元 0.21973219200979507
卸载 0.0
参数 0.0
发生器 0.0
变型 0.0
变速器 0.0
启动 0.0
吸收 0.0
命令 0.0
围绕 0.0
地向 0.0
坡道 0.10986609600489754
基准 0.0
增加 0.0
处于 0.05493304800244877
处在 0.0
大于 0.05493304800244877
大巴车 0.10986609600489754
头枕 0.0
安排 0.0
安装 0.0
定位 0.0
实施 0.0
实现 0.05493304800244877
容量 0.0
导引 0.0
小于 0.10986609600489754
峰值 0.0
布置 0.0
座椅 0.0
延伸 0.0
开启 0.0
开度 0.10986609600489754
引到 0.0
形成 0.0
总和 0.0
总成 0.0
恢复 0.05493304800244877
悬挂 0.0
成使 0.0
手刹 0.05493304800244877
手动 0.0
持续 0.05493304800244877
接到 0.0
控制 0.326886766389167

手动 0.0
持续 0.0
接到 0.0
控制 0.16733994339112407
控制系统 0.13123290920586314
描述 0.0
提供 0.0
摊开 0.0
操作 0.04880089213451652
支承 0.0
支撑 0.0
支架 0.0
放电 0.06561645460293157
数值 0.13123290920586314
整车 0.0
斜倚 0.0
方向 0.0
方式 0.0
方法 0.13016247043380072
方面 0.06561645460293157
旋转 0.0
无需 0.0
时间 0.0
更改 0.0
最低 0.06561645460293157
最靠近 0.0
机动 0.0
机动车 0.0
机动车辆 0.0
条件 0.0
构件 0.0
构造 0.11155996226074937
枢转 0.0
枢转地 0.0
枢轴 0.0
档位 0.0
检测 0.0
椅背 0.0
模式 0.0
横向 0.0
步骤 0.0
气囊 0.0
永磁 0.0
油门 0.0
泡沫 0.0
涉及 0.05577998113037468
添加 0.0
溃缩 0.0
满足 0.0
特别 0.0
状态 0.0
独立 0.13123290920586314
电动 0.0
电机 0.0
电池 0.5249316368234526
电池组 0.06561645460293157
电能 0.06561645460293157
监测 0.0
目标 0.0
目的 0.0
直到 0.0
相交 0.0
相关联 0.13123290920586314
着横 0.0
硬件 0.0
确定 0.13123290920586314
碰撞 0.0
移动 0.0
空挡 0.0
竖直 0.0
端部 0.0
策略 0.0
系统 0.43920802921064866
纵向 0.0
组件 0.0
织物 0.0
结合部 0.0
结构 0.0
继续 0.0
耦接 0.0
联接 0.0
膨胀 0.0
自动 0.0
行驶 0.0
表面 0.0
衬板 0.0
装置 0.0
装饰 0.0
覆盖 0.0
角度 0.0
触发 0.0
计算 0.19684936380879473
设备 0.0
设定值 0.0
设置 0.0
设计 0.0
评估 0.13123290920586314
负荷 0.0


货物 0.0
货箱 0.0
起步 0.0
越过 0.0
踏板 0.0
踩下 0.0
车厢 0.0
车身 0.0
车辆 0.0
车门 0.0
转速 0.0
轴线 0.0
辅助 0.0
输出 0.0
过程 0.17655390037464674
运行 0.08827695018732337
运输工具 0.0
进入 0.0
远离 0.0
退出 0.0
速度 0.08827695018732337
邻接 0.0
邻近 0.0
部上 0.0
部分 0.0
部向 0.0
配置 0.0
重叠 0.0
重启 0.08827695018732337
铰接 0.0
闭环控制 0.0
间隔 0.0
间隙 0.0
防止 0.0
防溜 0.0
降低 0.0
限制 0.0
限定 0.0
随后 0.08827695018732337
靠背 0.0
靠近 0.0
面板 0.0
驱动 0.06565417089546904
驱动器 0.0
驻车 0.0
-------这里输出第 10 篇文本的词语tf-idf------
一定 0.0
一段时间 0.0
主体 0.0
乘客 0.0
乘车 0.0
事件 0.0
二者 0.0
交叉 0.0
产品 0.045968414960952955
产生 0.0
介绍 0.0
仪表板 0.0
传感器 0.0
估计 0.0
位置 0.0
使用 0.0
使能 0.0
使该 0.0
信号 0.0
倾斜 0.5975893944923885
偏压 0.0
停转 0.0
储能 0.0
元件 0.0
关闭 0.0
具有 0.11203858206387621
内燃机 0.0
内管 0.0
凹陷 0.0
分析 0.0
分部 0.0
分隔 0.0
切换 0.0
利用 0.0
制动 0.0
制造 0.0
刹车 0.0
力矩 0.0
功能 0.0
包围 0.0
单元 0.0
卸载 0.13790524488285888
参数 0.0
发生器 0.0
变型 0.09193682992190591
变速器 0.0
启动 0.0
吸收 0.0
命令 0.0
围绕 0.0
地向 0.0
坡道 0.0
基准 0.0
增加 0.0
处于 0.0
处在 0.0
大于 0.0
大巴车 0.0
头枕 0.0
安排 0.0
安装 0.0
定位 0.0
实施 0.0
实现 0.0
容量 0.0
导引 0.0